In [1]:
import mne
import numpy as np
import os.path as op
from tools import files
from scipy.stats import trim_mean, ttest_1samp, sem
%matplotlib qt

In [2]:
trim = lambda x: trim_mean(x, 0.01, axis=0)

In [3]:
main_path = op.join("/cubric", "scratch", "c1557187", "act_mis")
print(op.exists(main_path))
meg_path = op.join(main_path, "MEG")
beh_path = op.join(main_path, "BEH")
res_means = op.join(main_path, "RESULTS", "means_erf_occ.npy")

True


In [4]:
participants = files.get_folders_files(meg_path, wp=False)[0]
participants.sort()
participants.remove("0042")
participants.remove("0004")
participants.remove("0009")
print(participants)

['0001', '0003', '0008', '0014', '0015', '0016', '0017', '0019', '0021', '0027', '0030', '0032', '0034', '0035', '0038', '0039', '0040', '0041', '0043']


In [5]:
cond = ["reg", "odd", "diff"]
data = {i: [] for i in cond}

In [6]:
for pp in participants:
    participant_path = op.join(meg_path, pp)
    participant_files = files.get_files(participant_path, "ica-obs-", "-epo.fif")[2]
    for i in participant_files:
        epochs = mne.read_epochs(i, preload=True)
        reg = epochs["30"].average(method=trim)
        odd = epochs["40"].average(method=trim)
        diff = mne.combine_evoked([-reg, odd], weights="equal")

        data["reg"].append(reg.data)
        data["odd"].append(odd.data)
        data["diff"].append(diff.data)

In [7]:
np.save(res_means, data)

# ANALYSIS

In [8]:
data = np.load(res_means)
data = data.item()

In [11]:
epo = "/cubric/scratch/c1557187/act_mis/MEG/0016/raw-000-raw.fif"
info = mne.io.read_raw_fif(epo, preload=True).pick_types(meg=True, eeg=True, ref_meg=False).info

In [12]:
diff = np.array(data["diff"])
diff_mean = trim_mean(diff, 0.01, axis=0)
diff_evo = mne.EvokedArray(diff_mean, info, tmin=-0.1)
diff_evo = diff_evo.pick_types(eeg=False)

In [13]:
# diff_evo.info["bads"] = bads
diff_evo.plot_joint();

In [15]:
reg = np.array(data["reg"])
reg_mean = trim_mean(reg, 0.01, axis=0)
reg_evo = mne.EvokedArray(reg_mean, info, tmin=-0.1)
reg_evo = reg_evo.pick_types(eeg=False)

In [16]:
# reg_evo.info["bads"] = bads
reg_evo.plot_joint();

In [18]:
odd = np.array(data["odd"])
odd_mean = trim_mean(odd, 0.01, axis=0)
odd_evo = mne.EvokedArray(odd_mean, info, tmin=-0.1)
odd_evo = odd_evo.pick_types(eeg=False)

In [19]:
# odd_evo.info["bads"] = bads
odd_evo.plot_joint();